In [3]:
import keras

In [ ]:
VGG19 = keras.applications.VGG19(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

574710816/574710816 ━━━━━━━━━━━━━━━━━━━━ 24s 0us/step


<Functional name=vgg19, built=True>

In [ ]:
import tensorflow as tf
import os

IMG_SIZE = 256
BATCH_SIZE = 4


def load_and_preprocess(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    return img


def make_dataset(dir_path):
    files = tf.data.Dataset.list_files(os.path.join(dir_path, "*"), shuffle=True)
    return files.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)


# --- TRAIN ---
train_style_ds = make_dataset("data/vangogh2photo/trainA")
train_content_ds = make_dataset("data/vangogh2photo/trainB")

train_ds = tf.data.Dataset.zip((train_content_ds, train_style_ds))
train_ds = train_ds.shuffle(500).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# --- TEST ---
test_content_ds = make_dataset("data/vangogh2photo/testA")
test_style_ds = make_dataset("data/vangogh2photo/testA")

test_ds = tf.data.Dataset.zip((test_content_ds, test_style_ds))
test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [12]:
import tensorflow as tf
import keras
from keras import layers

# Select layers for style + content — minimal example


def preprocess(image):
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = keras.applications.vgg19.preprocess_input(image * 255.0)
    return image


# Gram matrix for style
def gram_matrix(tensor):
    # tensor shape: [batch, height, width, channels]
    batch, h, w, c = tf.unstack(tf.shape(tensor))
    features = tf.reshape(tensor, [batch, h * w, c])
    gram = tf.matmul(features, features, transpose_a=True)
    return gram / tf.cast(h * w * c, tf.float32)


class StyleTransferNet(keras.Model):
    """
    alpha -- content preservation
    beta  -- style preservation
    """

    def __init__(self, train_ds, test_ds, alpha=1e4, beta=1e-2):
        super().__init__() 
        
        self.alpha = alpha
        self.beta = beta
        self.train_ds = train_ds
        self.test_ds = test_ds
        
        vgg = keras.applications.VGG19(
            include_top=False,
            weights="imagenet",
        )
        vgg.trainable = False
        
        content_layer = "block5_conv2"
        style_layers = ["block1_conv1", "block2_conv1", "block3_conv1", "block4_conv1"]

        vgg_layers = [
            vgg.get_layer(name).output for name in (style_layers + [content_layer])
        ]

        self.vgg = keras.Model(
            [vgg.input],
            outputs=vgg_layers,
        )

    def _style_loss(style_outputs, target_style_grams):
        loss = 0.0
        for output, target in zip(style_outputs, target_style_grams):
            loss += tf.reduce_mean(tf.square(gram_matrix(output) - target))
        return loss / float(len(style_outputs))

    def _content_loss(content_output, target_content):
        return tf.reduce_mean(tf.square(content_output - target_content))

    # Full loss
    def loss(self, model, content, style, generated):
        c = preprocess(content)
        s = preprocess(style)
        g = preprocess(generated)

        style_targets = self.vgg(s)[:-1]
        content_target = self.vgg(c)[-1]

        gen_outputs = self.vgg(g)
        gen_style_outputs = gen_outputs[:-1]
        gen_content_output = gen_outputs[-1]

        s_loss = self._style_loss(
            gen_style_outputs, [gram_matrix(t) for t in style_targets]
        )
        c_loss = self._content_loss(gen_content_output, content_target)

        return self.alpha * c_loss + self.beta * s_loss

    # Optimization step (e.g., using tf.GradientTape)
    @tf.function
    def train_step(self, generated, content_image, style_image, optimizer):
        with tf.GradientTape() as tape:
            loss = self.loss(content_image, style_image, generated)
        grads = tape.gradient(loss, generated)
        optimizer.apply_gradients([(grads, generated)])

        return loss

    def train(self, epochs):
        optimizer = tf.optimizers.Adam(learning_rate=0.02)
        
        for content_img, style_img in self.train_ds:
            generated = tf.Variable(
                tf.image.convert_image_dtype(content_img, tf.float32)
            )

            for i in range(epochs):
                loss = self.train_step(generated, content_img, style_img, optimizer)
                if i % 100 == 0:
                    print("Step:", i, "Loss:", loss)


In [ ]:
model = StyleTransferNet(train_ds=train_ds, test_ds=test_ds)

model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, None, None, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, None, None, 64) │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, None, None, 64) │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, None, None, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, None, None,     │        73,856 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, None, None,     │       147,584 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, None, None,     │       295,168 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv4 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, None, None,     │     1,180,160 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv4 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │             

 Total params: 15,304,768 (58.38 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 15,304,768 (58.38 MB)